In [1]:
import Pkg; Pkg.activate("../"); # Activate the project 
# Import all the necessary libraries
libraries = ["QuTaM", "LinearAlgebra", "Plots", "LaTeXStrings", "Statistics", "OrdinaryDiffEq", "ProgressMeter", "CSV", "DataFrames"]

function ensure_imports(packages::Vector{String})
    for pkg in packages
        try
            @eval using $(Symbol(pkg))
        catch e
            if e isa ArgumentError && occursin(pkg, e.msg)
                println("Installing $pkg...")
                Pkg.add(pkg)
                @eval using $(Symbol(pkg))
            else
                rethrow(e)
            end
        end
    end
end

ensure_imports(libraries)

  Activating project at `~/Documents/Research/MonitoringMetrology/QuTaM.jl`


In [45]:
rf_EPS = 1e-5 # Tolerance for the distance respect to the Frobenious norm
rf_delta= 0.0 # Detuning. For comparison with the Analytic WTD this must be zero
rf_omega = 0.5 # Rabi Frequency
rf_gamma = 0.5
rf_L = sqrt(rf_gamma)*QuTaM.sigma_m
rf_psi0 = zeros(ComplexF64, 2)
rf_psi0[1] = 1 # Initial condition

alpha = 10.0
L = sqrt(rf_gamma)*QuTaM.sigma_m + alpha*I
H = QuTaM.rf_H - 0.5im*(conj(alpha)*L-alpha*adjoint(L))

sys = System(H,[L]) #Jump Operators
params = SimulParameters(rf_psi0,
    25.0, # Final time. Set very long so that all trajectories jump
    1, # seed
    250, # Number of trajectories
    25000, # Number of samples in the finegrid
    1.0, # Multiplier to use in the fine grid
    1e-3 # Tolerance for passing Dark state test
)

SimulParameters(psi0=ComplexF64[1.0 + 0.0im, 0.0 + 0.0im]
nsamples=25000
seed=1
ntraj=250)
multiplier=1.0
tf=25.0
dt=0.001
eps=0.001)

In [46]:
t0 = eps(Float64) # To avoid having jumps at 0
ts = collect(LinRange(t0, params.multiplier*params.tf, params.nsamples))
Qs = Array{ComplexF64}(undef, sys.NLEVELS, sys.NLEVELS, params.nsamples)
Vs = Array{ComplexF64}(undef, sys.NLEVELS, sys.NLEVELS, params.nsamples)
QuTaM.precompute!(sys, params.nsamples, ts, Qs, Vs)
# To store the data
# Create copies of the everything arrays, one for each thread
W = Array{Float64}(undef, params.nsamples,1 )
P = Array{Float64}(undef, sys.NCHANNELS, 1)
tid  = 1 #threadid()


1

In [47]:
function myfunc()
    t0 = eps(Float64) # To avoid having jumps at 0
    ts = collect(LinRange(t0, params.multiplier*params.tf, params.nsamples))
    Qs = Array{ComplexF64}(undef, sys.NLEVELS, sys.NLEVELS, params.nsamples)
    Vs = Array{ComplexF64}(undef, sys.NLEVELS, sys.NLEVELS, params.nsamples)
    QuTaM.precompute!(sys, params.nsamples, ts, Qs, Vs)
    # To store the data
    data = Vector{Trajectory}(undef, params.ntraj)
    # Create copies of the everything arrays, one for each thread
    W = Array{Float64}(undef, params.nsamples, 1)
    P = Array{Float64}(undef, sys.NCHANNELS, 1)
    tid  = 1
    QuTaM.run_single_trajectory(sys, params, W[:, tid], P[:, tid],
                            ts, Qs, Vs; seed = params.seed + 1)
 
end

myfunc (generic function with 1 method)

In [49]:
@time begin myfunc()
end 

  5.364917 seconds (65.61 M allocations: 7.824 GiB, 8.72% gc time)


2609-element Vector{DetectionClick}:
 DetectionClick(2.220446049250313e-16, 1)
 DetectionClick(0.013000520020801055, 1)
 DetectionClick(0.00200008000320035, 1)
 DetectionClick(0.010000400016000862, 1)
 DetectionClick(0.014000560022401118, 1)
 DetectionClick(0.07500300012000502, 1)
 DetectionClick(0.0450018000720031, 1)
 DetectionClick(0.009000360014400799, 1)
 DetectionClick(0.006000240009600606, 1)
 DetectionClick(2.220446049250313e-16, 1)
 DetectionClick(0.017000680027201308, 1)
 DetectionClick(0.010000400016000862, 1)
 DetectionClick(0.004000160006400478, 1)
 ⋮
 DetectionClick(0.018000720028801376, 1)
 DetectionClick(0.021000840033601566, 1)
 DetectionClick(0.003000120004800414, 1)
 DetectionClick(0.006000240009600606, 1)
 DetectionClick(0.014000560022401118, 1)
 DetectionClick(0.005000200008000542, 1)
 DetectionClick(0.008000320012800734, 1)
 DetectionClick(0.001000040001600286, 1)
 DetectionClick(0.006000240009600606, 1)
 DetectionClick(2.220446049250313e-16, 1)
 DetectionClick(0.

In [ ]:
trajectories = run_trajectories(sys, params);

In [ ]:
## Average Simulation ################3
# Now from each trajectory, generate the states at the given times
ntimes = size(t_given)[1]
sample = zeros(ComplexF64, sys.NLEVELS, ntimes, params.ntraj)
@time begin
for n in 1:params.ntraj
    sample[:, :, n] = QuTaM.evaluate_at_t(t_given, sample_clicks[n], sys,  params.psi0)
end

end 